In [2]:
class MyMonad m where
    (>>==) :: m a -> (a -> m b) -> m b
    return :: a -> m a

class MyApplicative f where
    pure :: a -> f a
    (<**>) :: f (a -> b) -> f a -> f b

class MyFunctor f where
    fmap :: (a -> b) -> f a -> f b
    

# for Either

In [3]:
-- data Either a b = Left a | Right b

instance MyFunctor (Either a) where
    fmap :: (b -> c) -> Either a b -> Either a c
    fmap f (Right x) = Right $ f x
    fmap f (Left x) = Left x


instance MyApplicative (Either a) where
    pure :: b -> Either a b
    pure = Right
    
    (<**>) :: Either a (b -> c) -> Either a b -> Either a c
    _ <**> Left x = Left x
    Left f <**> _ = Left f
    Right f <**> Right x = Right $ f x

instance MyMonad (Either a) where
    return :: b -> Either a b
    return = pure

    (>>==) :: Either a b -> (b -> Either a c) -> Either a c
    Left x >>== f = Left x
    Right x >>== f = f x

-- fmap (1+) (Left 3)

-- Left (+1) <**> Right 2



# For [ ]

In [4]:
instance MyFunctor [] where
    fmap :: (a -> b) -> [a] -> [b]
    fmap = map

instance MyApplicative [] where
    pure :: a -> [a]
    pure x = [x]

    (<**>) :: [a -> b] -> [a] -> [b]
    [] <**> _ = []
    -- (f:fs) <**> xs = map f xs ++ fs <**> xs
    (f:fs) <**> xs = [f x | x <- xs, f <- fs]



instance MyMonad [] where
    return :: a -> [a]
    return = pure

    (>>==) :: [a] -> (a -> [b]) -> [b]
    [] >>== _ = []
    (x:xs) >>== f = f x ++ (xs >>== f)

# For (,)

In [5]:
instance MyFunctor ((,) a) where
    fmap :: (b -> c) -> (a,b) -> (a,c)
    fmap f (a,b) = (a, f b)

instance Monoid a => MyApplicative ((,) a) where
    pure :: b -> (a,b)
    pure b = (mempty,b)

    (<**>) :: (a, b -> c) -> (a,b) -> (a,c)
    (x, f) <**> (y, n) = (x <> y, f n)

instance Monoid a => MyMonad ((,) a) where 
    return = pure
    -- (>>==) :: (a,b) -> (b -> (a,c)) -> (a,c)
    -- (a,b) >>== f = 


# For (->)

In [6]:
instance MyFunctor ((->) a) where
    -- fmap :: (b -> c) -> ((->) a b) -> ((->) a c)
    fmap :: (b -> c) -> (a -> b) -> (a -> c)
    fmap = (.)

instance MyApplicative ((->) r) where
    pure :: a -> (r -> a)
    pure x = \y -> x

    (<**>) :: (r -> (a -> b)) -> (r -> a) -> (r -> b)
    f <**> g = \r -> f r (g r)

instance MyMonad ((->) r) where
    return = pure
    
    (>>==) :: (r -> a) -> (a -> (r -> b)) -> (r -> b)
    f >>== g = \r -> (g $ f r) r

Line 8: Redundant lambda
Found:
pure x = \ y -> x
Why not:
pure x y = xLine 8: Use const
Found:
\ y -> x
Why not:
const x